In [1]:
from crystalbuilder import *
import crystalbuilder.geometry as geo
import matplotlib.pyplot as plt
import tidy3d as td
import crystalbuilder.convert as cv
import numpy as np
import vedo

Here we will create a simulation of the core-shell diamond crystal. This is a rod-connected diamond with a cubic conventional cell. We'll set it up in the usual manner, but we can use an actual distance in nanometers for our lattice constant. `a_mag` is this constant in microns (Tidy3D default units)

In [2]:
lattice_constant = 500 #nm

a1 = [1, 0, 0]
a2 = [0, 1 ,0]
a3 = [0, 0, 1]

a_mag = lattice_constant/1000 #convert to microns
geo_lattice = lattice.Lattice(a1, a2, a3, magnitude = [a_mag, a_mag, a_mag])

We'll load our points from Bilbao (or a local copy if already done before). We don't scale any of this using the lattice constant.  

In [3]:
diamond = bilbao.SpaceGroup(227)
pt = 1/8
points = diamond.calculate_points(point_list=[(pt, pt, pt)]) #This is the 8a wyckoff position

The IUCr tabulated general positions are in units of the lattice constant. After we connect the nearest neighbors, we convert to our micron units and scale the structure. This is done by passing `a_mag` to `NearestNeighbors()`. 

In [4]:
radius = .04 #microns

crystal_core = geo.NearestNeighbors(points, radius=radius, neighborhood_range=.45, a_mag=a_mag)

For our core-shell structure, we will just create a slightly larger (by `shell_radius`) set of connected rods. 

In [5]:
shell_radius = .02 #microns

crystal_shell = geo.NearestNeighbors(points, radius=(radius+shell_radius), neighborhood_range=.45, a_mag=a_mag)

The geometry is just a combination of these two lattices, **making sure to include the core *after* the shell**. In Tidy3D, items later in the list will override earlier ones.

In [6]:
geometry = [crystal_shell, crystal_core]

In [7]:
a1_reps = 8
a2_reps = 8
a3_reps = 4
crystal = geo_lattice.tile_geogeometry(geometry, a1_reps, a2_reps, a3_reps )
print(len(crystal[0]))
print(len(crystal[1]))

4096
4096


In [ ]:
scene = viewer.visualize(crystal[1], c='white', res=5)
viewer.add_to_visualizer(crystal[0], scene, c='red', alpha=.5, res=5)


# scene.show().close()

In [11]:
shell = cv.geo_to_tidy3d(crystal[1], 3, name="Shell")
core = cv.geo_to_tidy3d(crystal[0], 2, name="Core")

In [12]:
# create source
lda0 = 0.75  # wavelength of interest (length scales are micrometers in Tidy3D)
freq0 = td.C_0 / lda0  # frequency of interest

source = td.PointDipole(
    center=(0, 0, 0),  # position of the dipole
    source_time=td.GaussianPulse(freq0=freq0, fwidth=freq0 / 10.0),  # time profile of the source
    polarization="Ey",  # polarization of the dipole
) # type: ignore



sim = td.Simulation(
    size=(15, 15, 8),  # simulation domain size
    grid_spec=td.GridSpec.auto(
        min_steps_per_wvl=20
    ),  # automatic nonuniform FDTD grid with 25 grids per wavelength in the material
    structures=[shell, core],
    sources=[source],
    run_time=3e-13,  # physical simulation time in second
) # type: ignore
sim.plot_3d()





In [13]:
from tidy3d import web

task_id = web.upload(sim, task_name=f"{a1_reps}x{a2_reps}x{a3_reps}-CoreShell")

19:59:34 Central Daylight Time Created task '8x8x4-CoreShell' with task_id      
                               'fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770' and  
                               task_type 'FDTD'.

                               View task using web UI at                        
                               ]8;id=850338;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770\'https://tidy3d.simulation.cloud/workbench?]8;;\]8;id=561981;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770\taskId]8;;\
                               ]8;id=850338;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770\=]8;;\]8;id=332714;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770\fdve]8;;\]8;id=850338;https://tidy3d.simulation.cloud/workbench?taskId=fdve-21c33ec8-8538-4d7c-99ac-5e10f7ab7770\-21c33ec8-8538-4d7c-99ac-5e10f7ab7770']8;;\.

                               Task folder: ]8;id=747560;https://tidy3d.simulation.cloud/folders/767b270d-f545-4c6b-9509-691c0e60105a\'default']8;;\.

c:\Users\Brandon\miniconda3\envs\simulation\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

19:59:51 Central Daylight Time Maximum FlexCredit cost: 0.220. Minimum cost     
                               depends on task execution details. Use           
                               'web.real_cost(task_id)' to get the billed       
                               FlexCredit cost after a simulation run.